# Load Libraries

In [4]:
import csv
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid
from sklearn.ensemble import RandomForestClassifier

# Import Data

In [5]:
X_train = pd.read_csv('0_X_train.csv', index_col='Id')
X_valid = pd.read_csv('1_X_valid.csv', index_col='Id')
X_test  = pd.read_csv('2_X_test.csv', index_col='Id')

y_train = pd.read_csv('0_y_train.csv', index_col='Id')
y_valid = pd.read_csv('1_y_valid.csv', index_col='Id')
y_test  = pd.read_csv('2_y_test.csv', index_col='Id')

X_pred = pd.read_csv("TEST_KAGGLE.csv", index_col="Id")

# Bagging

## Find best configuration

In [6]:
bag_hyperparam_grid={'min_samples_leaf':[1, 3, 5, 7, 9], "n_estimators":[500, 750, 1000, 1250, 1500, 2000]}
bag = RandomForestClassifier(oob_score=True, max_features=None,
                             warm_start=False, random_state=1, n_jobs=-2)
best_score=0.5

for g in ParameterGrid(bag_hyperparam_grid):
    bag.set_params(**g)
    #or rfc.set_params(min_samples_leaf=g['min_samples_leaf'], max_features=g['max_features'])
    bag.fit(X_train,y_train)
    # save if best
    if bag.oob_score_ > best_score:
        best_score = bag.oob_score_
        best_params_bag = g

print(f"OOB: %0.5f" % best_score)
print("Best parameters:", best_params_bag)

C:\Users\ficop\AppData\Local\Temp\ipykernel_23096\1052585566.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  bag.fit(X_train,y_train)
C:\Users\ficop\AppData\Local\Temp\ipykernel_23096\1052585566.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  bag.fit(X_train,y_train)
C:\Users\ficop\AppData\Local\Temp\ipykernel_23096\1052585566.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  bag.fit(X_train,y_train)
C:\Users\ficop\AppData\Local\Temp\ipykernel_23096\1052585566.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  bag.fit(X_train,y_train)
C:\U

OOB: 0.85142
Best parameters: {'min_samples_leaf': 3, 'n_estimators': 1500}


## Generate Bagged Tree

In [7]:
bag = RandomForestClassifier(n_estimators=best_params_bag["n_estimators"], max_features=None,
                             min_samples_leaf=best_params_bag['min_samples_leaf'],
                             oob_score=True, warm_start=False, random_state=1)

bag.fit(X_train, y_train)

y_valid_pred = bag.predict(X_valid)
y_test_pred = bag.predict(X_test)

C:\Users\ficop\AppData\Local\Temp\ipykernel_23096\3665481828.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  bag.fit(X_train, y_train)


## Test Results

In [8]:
print(bag.oob_score_)
print(bag.score(X_valid, y_valid))
print(bag.score(X_test, y_test))

0.8514203638684966
0.8309754281459419
0.849590469099032


# Random Forest

## Find best Configuration

In [9]:
hyperparam_grid={"max_features":[2, 3, 4, 5, 6, 7, 8],
                 'min_samples_leaf':[1, 3, 5, 7, 9, 11, 13],
                 "n_estimators":[500, 1000, 2000, 3000, 4000, 5000]}

rfc = RandomForestClassifier(oob_score=True, warm_start=False, random_state=1, n_jobs=-2)
best_score=0.5

for g in ParameterGrid(hyperparam_grid):
    rfc.set_params(**g)
    #or rfc.set_params(min_samples_leaf=g['min_samples_leaf'], max_features=g['max_features'])
    rfc.fit(X_train,y_train)
    # save if best
    if rfc.oob_score_ > best_score:
        best_score = rfc.oob_score_
        best_params = g

print(f"OOB: %0.5f" % best_score)
print("Best parameters:", best_params)

C:\Users\ficop\AppData\Local\Temp\ipykernel_23096\1936959364.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train,y_train)
C:\Users\ficop\AppData\Local\Temp\ipykernel_23096\1936959364.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train,y_train)
C:\Users\ficop\AppData\Local\Temp\ipykernel_23096\1936959364.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train,y_train)
C:\Users\ficop\AppData\Local\Temp\ipykernel_23096\1936959364.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train,y_train)


OOB: 0.85381
Best parameters: {'max_features': 8, 'min_samples_leaf': 1, 'n_estimators': 1000}


## Generate Random Forest

In [10]:
rfc = RandomForestClassifier(n_estimators=best_params["n_estimators"], 
                             max_features=best_params['max_features'],
                             min_samples_leaf=best_params['min_samples_leaf'], 
                             criterion="entropy", oob_score=True, warm_start=False, random_state=1)

rfc.fit(X_train, y_train)

y_valid_pred = rfc.predict(X_valid)
y_test_pred = rfc.predict(X_test)

y_valid_proba = rfc.predict_proba(X_valid)
y_test_proba = rfc.predict_proba(X_test)


C:\Users\ficop\AppData\Local\Temp\ipykernel_23096\2986525241.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train, y_train)


## Test Results

In [11]:
print(rfc.oob_score_)
print(rfc.score(X_valid, y_valid))
print(rfc.score(X_test, y_test))

0.8525375039897861
0.8376768428890543
0.849590469099032


In [12]:
y_pred = rfc.predict(X_pred)

In [13]:
file = open('test_file_randForest.csv', 'w')
writer = csv.writer(file)
writer.writerow(['Id', 'subscription'])
for i in range(len(y_pred)):
    writer.writerow([i, y_pred[i]])
file.close()